# GPT-3.5-Turbo on GSM8K

In [16]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [39]:
openai.api_key = "sk-"

In [18]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('../gsm8k/lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [19]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [20]:
gsm8k_test = gsm8k['test']

In [21]:
prompt_complex = open('../gsm8k/lib_prompt/prompt_hardest.txt').read()

In [22]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

## prompt compression

In [28]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor('microsoft/phi-2', device_map="cpu")
compressed_prompt = llm_lingua.compress_prompt(prompt_complex, target_token=1000)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2207 > 2048). Running this sequence through the model will result in indexing errors


In [30]:
prompt_compressed = compressed_prompt["compressed_prompt"]

In [31]:
prompt_compressed

"Question: Angelo and Melanie to plan how many hours next should study together for their test next. They have 2 chapters of their textbook to study and 4ets to. They out that they should dedicate 3 hours to each chapter of and 1 hours. they than 4 they week they a hour include 3minute and\nLet's think step\n Melanie think should each of chapters, 3 hours 2 hours total\n theets they hours each,\n Melanie to start with planning 12 hours to study, at 4 hours a day, 12 3 days.\n, they need to include for breaks and lunch. Every hour they to include a 10 break, so 12 total hours x minutes 120 minutes breaks.\nThey to include 3minute breaks, 3 30 minutes\nAnd to include lunch day, so 120 minutes for minutes, or 180 60 minutes 3 hours\nSo Melanie to plan hours study 3 hours breaks hours total\nThey to study than hours, 15 hours hours75\nThey will to plan study 4 days to for all the time they need\nThe is 4\n\nQuestion: Mark's team scores 25 2 pointers, 3 pointers and 10 throws.  Their oppone

In [32]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [33]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [35]:
prompt_q1 = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'
prompt_q2 = prompt_compressed + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [43]:
response1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q1},
    ]
)

response2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q2},
    ]
)

In [41]:
response1['choices'][0]['message']['content']

'It takes 2 bolts of blue fiber and half that much white fiber, which is 2/2 = 1 bolt.\nTherefore, the robe takes 2 blue bolts + 1 white bolt = 3 bolts in total. Answer: 3.'

In [44]:
response2['choices'][0]['message']['content']

'Step 1: A robe takes 2 bolts of blue fiber. So, blue bolts = 2 bolts.\nStep 2: It also takes half that much white fiber. So, white bolts = 2 bolts / 2 = 1 bolt.\nStep 3: Total bolts needed = blue bolts + white bolts = 2 bolts + 1 bolt = 3 bolts.\nTherefore, it takes a total of 3 bolts to make the robe.'

# Complex Prompt Random Sampling, Acc 77.1

### 1) complex

In [53]:
i = 0
with open('./outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 10): break

  1%|▎                                    | 9/1319 [00:32<1:19:58,  3.66s/it]


In [57]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

num_q 10 correct 7 ratio 0.7000


### 2) compressed

In [55]:
i = 0
with open('./outputs/test_gpt_3.5_turbo_compressed.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_compressed + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 10): break

  1%|▎                                    | 9/1319 [00:35<1:25:25,  3.91s/it]


In [56]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_compressed.txt')

num_q 10 correct 5 ratio 0.5000


# Complex Prompt Greedy Decoding, Acc 78.85

### 1) complex

In [58]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 10): break

  1%|▎                                    | 9/1319 [00:33<1:21:18,  3.72s/it]


In [59]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_temp_0.txt')

num_q 10 correct 6 ratio 0.6000


### 2) compressed

In [61]:
i = 0
with open('outputs/test_gpt_3.5_turbo_compressed_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_compressed + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 10): break

  1%|▎                                    | 9/1319 [01:24<3:24:06,  9.35s/it]


In [62]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_compressed_temp_0.txt')

num_q 10 correct 5 ratio 0.5000


# Baseline Prompt Greedy Decoding, Acc 74.98

### 1) prompt_original

In [22]:
prompt_original = open('../gsm8k/lib_prompt/prompt_original.txt').read()

In [23]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        if(i == 10): break

  0%|                                                            | 2/1319 [00:07<1:18:19,  3.57s/it]


In [24]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_temp_0.txt')

num_q 3 correct 2 ratio 0.6667


# Baseline Prompt, Dialog In-Context Learning, Acc 76.8

In [24]:
def make_dialog_prompt(prompt):
    messages = []
    messages.append({"role": "system", "content": "Follow the given examples and answer the question."})
    cases = prompt.split("\n\n")
    for c in cases[:-1]:
        question = c.split("\n")[:2]
        messages.append({"role": "user", "content": "\n".join(question)})
        answer = c.split("\n")[2:]
        messages.append({"role": "assistant", "content": "\n".join(answer)})
    messages.append({"role": "user", "content": cases[-1] + "Let's think step by step"})
    return messages

In [30]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:57:13<00:00,  8.06s/it]


In [31]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt')

num_q 1319 correct 1013 ratio 0.7680


# Complex Prompt, Dialog In-Context Learning, Acc 

In [34]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:43:01<00:00,  7.42s/it]


In [35]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt')

num_q 1319 correct 988 ratio 0.7491


### prompt compression

In [3]:
prompt = """
Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want to include a 10-minute break, so 12 total hours x 10 minutes = 120 extra minutes for breaks.
They also want to include 3 10-minute snack breaks, 3 x 10 minutes = 30 minutes.
And they want to include 30 minutes for lunch each day, so 120 minutes for breaks + 30 minutes for snack breaks + 30 minutes for lunch = 180 minutes, or 180 / 60 minutes per hour = 3 extra hours.
So Angelo and Melanie want to plan 12 hours to study + 3 hours of breaks = 15 hours total.
They want to study no more than 4 hours each day, 15 hours / 4 hours each day = 3.75
They will need to plan to study 4 days to allow for all the time they need.
The answer is 4

Question: Mark's basketball team scores 25 2 pointers, 8 3 pointers and 10 free throws.  Their opponents score double the 2 pointers but half the 3 pointers and free throws.  What's the total number of points scored by both teams added together?
Let's think step by step
Mark's team scores 25 2 pointers, meaning they scored 25*2= 50 points in 2 pointers.
His team also scores 6 3 pointers, meaning they scored 8*3= 24 points in 3 pointers
They scored 10 free throws, and free throws count as one point so they scored 10*1=10 points in free throws.
All together his team scored 50+24+10= 84 points
Mark's opponents scored double his team's number of 2 pointers, meaning they scored 50*2=100 points in 2 pointers.
His opponents scored half his team's number of 3 pointers, meaning they scored 24/2= 12 points in 3 pointers.
They also scored half Mark's team's points in free throws, meaning they scored 10/2=5 points in free throws.
All together Mark's opponents scored 100+12+5=117 points
The total score for the game is both team's scores added together, so it is 84+117=201 points
The answer is 201

Question: Bella has two times as many marbles as frisbees. She also has 20 more frisbees than deck cards. If she buys 2/5 times more of each item, what would be the total number of the items she will have if she currently has 60 marbles?
Let's think step by step
When Bella buys 2/5 times more marbles, she'll have increased the number of marbles by 2/5*60 = 24
The total number of marbles she'll have is 60+24 = 84
If Bella currently has 60 marbles, and she has two times as many marbles as frisbees, she has 60/2 = 30 frisbees.
If Bella buys 2/5 times more frisbees, she'll have 2/5*30 = 12 more frisbees.
The total number of frisbees she'll have will increase to 30+12 = 42
Bella also has 20 more frisbees than deck cards, meaning she has 30-20 = 10 deck cards
If she buys 2/5 times more deck cards, she'll have 2/5*10 = 4 more deck cards.
The total number of deck cards she'll have is 10+4 = 14
Together, Bella will have a total of 14+42+84 = 140 items
The answer is 140

Question: A group of 4 fruit baskets contains 9 apples, 15 oranges, and 14 bananas in the first three baskets and 2 less of each fruit in the fourth basket. How many fruits are there?
Let's think step by step
For the first three baskets, the number of apples and oranges in one basket is 9+15=24
In total, together with bananas, the number of fruits in one basket is 24+14=38 for the first three baskets.
Since there are three baskets each having 38 fruits, there are 3*38=114 fruits in the first three baskets.
The number of apples in the fourth basket is 9-2=7
There are also 15-2=13 oranges in the fourth basket
The combined number of oranges and apples in the fourth basket is 13+7=20
The fourth basket also contains 14-2=12 bananas.
In total, the fourth basket has 20+12=32 fruits.
The four baskets together have 32+114=146 fruits.
The answer is 146

Question: You can buy 4 apples or 1 watermelon for the same price. You bought 36 fruits evenly split between oranges, apples and watermelons, and the price of 1 orange is $0.50. How much does 1 apple cost if your total bill was $66?
Let's think step by step
If 36 fruits were evenly split between 3 types of fruits, then I bought 36/3 = 12 units of each fruit
If 1 orange costs $0.50 then 12 oranges will cost $0.50 * 12 = $6
If my total bill was $66 and I spent $6 on oranges then I spent $66 - $6 = $60 on the other 2 fruit types.
Assuming the price of watermelon is W, and knowing that you can buy 4 apples for the same price and that the price of one apple is A, then 1W=4A
If we know we bought 12 watermelons and 12 apples for $60, then we know that $60 = 12W + 12A
Knowing that 1W=4A, then we can convert the above to $60 = 12(4A) + 12A
$60 = 48A + 12A
$60 = 60A
Then we know the price of one apple (A) is $60/60= $1
The answer is 1

Question: Susy goes to a large school with 800 students, while Sarah goes to a smaller school with only 300 students.  At the start of the school year, Susy had 100 social media followers.  She gained 40 new followers in the first week of the school year, half that in the second week, and half of that in the third week.  Sarah only had 50 social media followers at the start of the year, but she gained 90 new followers the first week, a third of that in the second week, and a third of that in the third week.  After three weeks, how many social media followers did the girl with the most total followers have?
Let's think step by step
After one week, Susy has 100+40 = 140 followers.
In the second week, Susy gains 40/2 = 20 new followers.
In the third week, Susy gains 20/2 = 10 new followers.
In total, Susy finishes the three weeks with 140+20+10 = 170 total followers.
After one week, Sarah has 50+90 = 140 followers.
After the second week, Sarah gains 90/3 = 30 followers.
After the third week, Sarah gains 30/3 = 10 followers.
So, Sarah finishes the three weeks with 140+30+10 = 180 total followers.
Thus, Sarah is the girl with the most total followers with a total of 180.
The answer is 180

Question: Sam bought a dozen boxes, each with 30 highlighter pens inside, for $10 each box. He rearranged five of these boxes into packages of six highlighters each and sold them for $3 per package. He sold the rest of the highlighters separately at the rate of three pens for $2. How much profit did he make in total, in dollars?
Let's think step by step
Sam bought 12 boxes x $10 = $120 worth of highlighters.
He bought 12 * 30 = 360 highlighters in total.
Sam then took 5 boxes × 6 highlighters/box = 30 highlighters.
He sold these boxes for 5 * $3 = $15
After selling these 5 boxes there were 360 - 30 = 330 highlighters remaining.
These form 330 / 3 = 110 groups of three pens.
He sold each of these groups for $2 each, so made 110 * 2 = $220 from them.
In total, then, he earned $220 + $15 = $235.
Since his original cost was $120, he earned $235 - $120 = $115 in profit.
The answer is 115

Question: In a certain school, 2/3 of the male students like to play basketball, but only 1/5 of the female students like to play basketball. What percent of the population of the school do not like to play basketball if the ratio of the male to female students is 3:2 and there are 1000 students?
Let's think step by step
The students are divided into 3 + 2 = 5 parts where 3 parts are for males and 2 parts are for females.
Each part represents 1000/5 = 200 students.
So, there are 3 x 200 = 600 males.
And there are 2 x 200 = 400 females.
Hence, 600 x 2/3 = 400 males play basketball.
And 400 x 1/5 = 80 females play basketball.
A total of 400 + 80 = 480 students play basketball.
Therefore, 1000 - 480 = 520 do not like to play basketball.
The percentage of the school that do not like to play basketball is 520/1000 * 100 = 52
The answer is 52

Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
"""

In [ ]:
"""
There are 2 roles:
 1) Math Teacher
 2) History Teacher

You'll be given a question, and your job is to answer which one of the roles above would answer the question better.
Your answer should only be 'Math Teacher' or 'History Teacher', no other explanations!

<Example>
Example 1) 
[Q] What is the Newton’s law? 
[A] Math Teacher

Example 2)
[Q] How did World War I end?
[A] History Teacher
</Example>

Q) Who is the world best mathmatician in 21st century?
A) 

"""

In [ ]:
"""
There are 2 roles:
 1) Math Teacher
 2) History Teacher

You'll be given a question, and your job is figure out a better role to answer that question.
After choosing the better role, answer the question pretending to be that role!
Your answer format should be something like this: 
'''
<role>: <answer>
'''

Let's think step by step

1. example question) What is the Newton's law?
2. you should choose which role(math teacher / history teacher) would answer the input question better. In this case, a 'Math teacher' would better answer the question related to newton's law.
3. Pretending to be a math teacher(the role you chose in step 2), answer the input question: 'What is the Newton's law?'
4. Your answer should be something like this: <Math Teacher>: <the actual answer>


Q) When did world war 2 happen?
A) 
"""